In [1]:
import numpy as np
import pandas as pd

In [8]:
import pandas as pd    

with open('data/GSS.dat','r') as f:
    next(f) # skip first row
    df2 = pd.DataFrame(l.rstrip().split() for l in f)

# print(df)

In [6]:
df = pd.read_csv('data/GSS.csv')

In [7]:
df.head()

,YEAR,ID_,WRKSTAT,HRS1,HRS2,EVWORK,WRKSLF,WRKGOVT,OCC10,INDUS10,...,FEHIRE,RELPERSN,SPRTPRSN,OTHLANG,OTHLANG1,OTHLANG2,MEOVRWRK,DATEINTV,COHORT,BALLOT
0,1972,1,1,-1,-1,0,2,0,520,5170,...,0,0,0,0,0,0,0,0,1949,0
1,1972,2,5,-1,-1,1,2,0,7700,6470,...,0,0,0,0,0,0,0,0,1902,0
2,1972,3,2,-1,-1,0,2,0,4920,7070,...,0,0,0,0,0,0,0,0,1924,0
3,1972,4,1,-1,-1,0,2,0,800,5170,...,0,0,0,0,0,0,0,0,1945,0
4,1972,5,7,-1,-1,1,2,0,5020,6680,...,0,0,0,0,0,0,0,0,1911,0


In [13]:
# df.dtypes

In [ ]:
[hrs1,hrs2,sibs,childs,age,]

In [14]:
print(df.filter(like='spike').columns)

TypeError: 'int' object is not iterable

In [ ]:
df.hrs1

In [ ]:
pd.as

In [18]:
df2 =df.apply(str)

In [29]:
df2 = pd.DataFrame()

for col in df:
    df2[col] = df[col].apply(str)

In [32]:
df2.dtypes.head()

YEAR       object
ID_        object
WRKSTAT    object
HRS1       object
HRS2       object
dtype: object

In [ ]:
df2.apply(lambda )

In [39]:
True in list(df2.YEAR.str.contains('1998'))

True